In [ ]:
!pip install Pillow
!pip install torch torchvision torchmetrics pytorch_lightning
!pip install matplotlib
!pip install wandb

In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Currently logged in as: aidan-lw17 (remmie-uoft). Use `wandb login --relogin` to force relogin


In [ ]:
# General imports.
import math

from PIL import Image

import torch
import torchmetrics
import torchvision.datasets
import torchvision.transforms
import torchvision.models as models
import torch.utils.data
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

import matplotlib.pyplot as plt

# Enable reproducible results.
torch.manual_seed(42)

/opt/anaconda3/envs/om-classifier/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import wandb

In [ ]:
# The provided dataset consists of rectangular images.
# These contain a label in the bottom left, and a central circular image.
# Throw away a bit to make sure the borders are completely gone.
# We are conservative and throw away a bit to make sure the borders are completely gone.
CIRCULAR_IMAGE_RADIUS = int(750 * 0.95)

# However, it's far easier to work with square images.
# So, for now, we simply consider the largest square inscribed in the circle.
SQUARE_IMAGE_WIDTH = (2 * CIRCULAR_IMAGE_RADIUS) / math.sqrt(2)

TRAIN_SPLIT = 0.6 #@param
VAL_SPLIT = 0.3 #@param

# When we load the dataset, it will contain both train and test.
# So, we only apply a limited set of transforms that should be used on both.
universal_transforms = torchvision.transforms.Compose([
  # The image naturally contains a border, which should be removed.
  torchvision.transforms.CenterCrop(SQUARE_IMAGE_WIDTH),

  # Autocontrast applies a somewhat adaptive contrast,
  # clamping the darkest pixel to black, the lightes to white, and interpolating.
  # We could use a manual contrast, but this risks overcontrasting some images
  # (what oversat is to white, overcontrast is to black)
  # Note that this is why we're conservative with the crop above, if any
  # of the black border remains, it will skew the clamp.
  torchvision.transforms.RandomAutocontrast(1),
  torchvision.transforms.PILToTensor(),
  torchvision.transforms.ConvertImageDtype(torch.float),
])

In [ ]:
dataset = torchvision.datasets.ImageFolder("./binary-data", transform=universal_transforms)

def show_image(dataset, number):
    plt.imshow(dataset[number][0].permute(1, 2, 0))

split_train = int(len(dataset) * TRAIN_SPLIT)
split_val = int(len(dataset) * VAL_SPLIT)
split_test = len(dataset) - split_train - split_val

dataset_train, dataset_val, dataset_test = torch.utils.data.random_split(dataset, [split_train, split_val, split_test], generator=torch.Generator().manual_seed(42))

In [ ]:
# Apply augmentation to training dataset.
# Note, although this looks like it is only applied once,
dataloader_train = torch.utils.data.DataLoader(dataset_train, 50, True)
dataloader_val = torch.utils.data.DataLoader(dataset_val, 1, False)

# This should be called to augment X during each training iteration.
# I.e, x' = apply_augmentation(x), pass x' to the actual model.
apply_augmentation = torchvision.transforms.Compose([
  torchvision.transforms.ConvertImageDtype(torch.uint8),
  torchvision.transforms.AutoAugment(),
  torchvision.transforms.ConvertImageDtype(torch.float)
])

In [ ]:
model_name = "densenet"
num_classes = 2
batch_size = 8
num_epochs = 15
feature_extract = True
IMG_SIZE = 1006
input_size = 224
momentum =

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
apply_augmentation_densenet = torchvision.transforms.Compose([
  torchvision.transforms.ConvertImageDtype(torch.uint8),
  torchvision.transforms.AutoAugment(),
  torchvision.transforms.ConvertImageDtype(torch.float),
  torchvision.transforms.RandomResizedCrop(input_size),
  torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
class DenseNetTransfer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        backbone = models.densenet121(pretrained=True)
        set_parameter_requires_grad(backbone, feature_extract)
        num_filters = backbone.classifier.in_features

        num_target_classes = 2
        backbone.classifier = nn.Linear(num_filters, num_target_classes)
        self.model = backbone
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()

        self.train_precision = torchmetrics.Precision(average='macro', num_classes=2,  multiclass = True)
        self.train_recall = torchmetrics.Recall(average='macro', num_classes=2,  multiclass = True)
        self.val_precision = torchmetrics.Precision(average='macro', num_classes=2,  multiclass = True)
        self.val_recall = torchmetrics.Recall(average='macro', num_classes=2,  multiclass = True)

    def forward(self, x):
        x = self.model(x)
        return x

    def loss_fn(self, out, target):
        return nn.CrossEntropyLoss()(out.view(-1, 2), target)

    def configure_optimizers(self):
        LR = 1e-2
        momentum = 0.01
        return torch.optim.SGD(self.parameters(), lr=LR, momentum=0.01)

    def training_step(self, batch, batch_idx):
        x, y = batch[0], batch[1]
        img = x.view(-1, 3, IMG_SIZE, IMG_SIZE)
        augmented_img = apply_augmentation_densenet(img)
        label = y.view(-1)
        out = self(augmented_img)
        loss = self.loss_fn(out, label)
#         print(f"Train loss: {loss}")
        self.train_acc.update(torch.argmax(out, dim=1), label)
        self.train_precision.update(out[:, 0], label)
        self.train_recall.update(out[:, 0], label)
#         acc = self.accuracy(torch.argmax(out, dim=1), label)
#         print(f"Accuracy: {acc}")
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch[0], batch[1]
        img = x.view(-1, 3, IMG_SIZE, IMG_SIZE)
        label = y.view(-1)
        out = self(apply_augmentation_densenet(img))
        loss = self.loss_fn(out, label)
#         print("=====================")
#         print(f"Loss: {loss}")
        out = nn.Softmax(-1)(out)
        logits = torch.argmax(out, dim=1)
        self.val_acc.update(logits, label)
        self.val_precision.update(out[:, 0], label)
        self.val_recall.update(out[:, 0], label)
        return loss

    def training_epoch_end(self, training_step_outputs):
        # compute metrics
        train_accuracy = self.train_acc.compute()
        train_precision = self.train_precision.compute()
        train_recall = self.train_recall.compute()
        train_loss = torch.stack([x['loss'] for x in training_step_outputs]).mean()

        # reset all metrics
        self.train_acc.reset()
        self.train_precision.reset()
        self.train_recall.reset()
        print(f"Training accuracy: {train_accuracy}")
        print(f"Training loss: {train_loss}")
        print(f"Train precision: {train_precision}")
        print(f"Train recall: {train_recall}")

        self.log("train/loss", train_loss)
        self.log("train/accuracy", train_accuracy)
        self.log("train/precision", train_precision)
        self.log("train/recall", train_recall)

    def validation_epoch_end(self, validation_step_outputs):
        print(f"VAL STEP OUTPUTS: {validation_step_outputs}")
        # compute metrics
        val_loss = torch.tensor(validation_step_outputs).mean()
        val_accuracy = self.val_acc.compute()
        val_precision = self.val_precision.compute()
        val_recall = self.val_recall.compute()

        # reset all metrics
        self.val_acc.reset()
        self.val_precision.reset()
        self.val_recall.reset()
        print("\n==========\n")
        print(f"Val accuracy: {val_accuracy}")
        print(f"Val loss: {val_loss}")
        print(f"Val precision: {val_precision}")
        print(f"Val recall: {val_recall}")

        self.log("val/loss", val_loss)
        self.log("val/accuracy", val_accuracy)
        self.log(f"val/precision", val_precision)
        self.log(f"val/recall", val_recall)


In [ ]:
wandb_logger = WandbLogger(project="DenseNet")
model = DenseNetTransfer()
trainer = pl.Trainer(limit_train_batches=50, max_epochs=50, logger=wandb_logger)
trainer.fit(model=model, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

/opt/anaconda3/envs/om-classifier/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name            | Type      | Params
----------------------------------------------
0 | model           | DenseNet  | 7.0 M 
1 | train_acc       | Accuracy  | 0     
2 | val_acc         | Accuracy  | 0     
3 | train_precision | Precision | 0     
4 | train_recall    | Recall    | 0     
5 | val_precision   | Precision | 0     
6 | val_recall      | Recall    | 0     
----------------------------------------------
2.0 K     Trainable params
7.0 M     Non-trainable params
7.0 M     Total params
27.824    Total estimated 

Sanity Checking DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.60it/s]VAL STEP OUTPUTS: [tensor(0.5042), tensor(0.7769)]


Val accuracy: 0.5
Val loss: 0.6405312418937683
Val precision: 0.5
Val recall: 0.25
                                                                                                                                                                       

/opt/anaconda3/envs/om-classifier/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   4%|███▋                                                                                          | 35/895 [06:21<2:36:09, 10.89s/it, loss=0.685, v_num=v058]
Validation: 0it [00:00, ?it/s]
Epoch 0:  14%|█████████████▎                                                                                 | 126/895 [06:40<40:46,  3.18s/it, loss=0.685, v_num=v058]


Epoch 0:  25%|███████████████████████▎                                                                       | 220/895 [07:01<21:33,  1.92s/it, loss=0.685, v_num=v058]


...

Epoch 2:  77%|███████████████████████████████████████████████████████████████████████▍                     | 687/895 [1:58:09<35:46, 10.32s/it, loss=0.661, v_num=v058]


Epoch 2:  87%|█████████████████████████████████████████████████████████████████████████████████▏           | 781/895 [1:58:31<17:18,  9.11s/it, loss=0.661, v_num=v058]


Epoch 2:  98%|██████████████████████████████████████████████████████████████████████████████████████████▉  | 875/895 [1:58:54<02:43,  8.15s/it, loss=0.661, v_num=v058]


Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 895/895 [1:58:59<00:00,  7.98s/it, loss=0.661, v_num=v058]VAL STEP OUTPUTS: [tensor(0.1332), tensor(0.1772), tensor(0.1426), tensor(2.3481), tensor(0.6555), tensor(0.7399), tensor(0.3603), tensor(0.0649), tensor(1.5557), tensor(0.3253), tensor(0.2029), tensor(0.3402), tensor(1.9614), tensor(0.0592), tensor(0.6729), tensor(0.9163), tensor(0.0421), tensor(1.8533), tensor(0.2144), tensor(1.0467), tensor(0.2720), tensor(0.9659), tensor(0.0970), tensor(0.1691), tensor(0.1816), tensor(0.5411), tensor(0.1688), tensor(0.1852), tensor(0.0396), tensor(0.1822), tensor(0.0578), tensor(0.1311), tensor(0.0479), tensor(0.1162), tensor(0.4997), tensor(0.7523), tensor(0.2293), tensor(0.0908), tensor(0.6272), tensor(0.3668), tensor(0.1280), tensor(1.4576), tensor(0.3125), tensor(0.0579), tensor(1.6444), tensor(1.0833), tensor(0.2412), tensor(0.8219), tensor(0.8974), tensor(0.4560), tensor(0.4903)

Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 895/895 [1:58:59<00:00,  7.98s/it, loss=0.661, v_num=v058]
                                                                                                                                                                       Training accuracy: 0.6751888394355774
Training loss: 0.6476882696151733
Train precision: 0.39280351996421814
Train recall: 0.46433791518211365
Epoch 3:   2%|█▉                                                                                            | 19/895 [03:27<2:39:07, 10.90s/it, loss=0.686, v_num=v058]

In [ ]:
import wandb
wandb.finish()

epoch,▁▁▃▃▆▆██
train/accuracy,▁▆▆█
train/loss,▄█▅▁
train/precision,▇▁▁█
train/recall,▅▂▁█
trainer/global_step,▁▁▃▃▆▆██
val/accuracy,██▇▁
val/loss,█▇▂▁
val/precision,▁█▅▅
val/recall,███▁
epoch,3
